<a href="https://colab.research.google.com/github/HsuLiSheng/big-data/blob/main/8_%E6%9B%B8%E5%BA%97%E6%96%B0%E6%9B%B8%E6%8E%92%E8%A1%8C%E6%A6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
###找每個分類中共有幾頁再每一頁查詢
###url:不同分類的網址, kind:分類的名稱
def showbook(url, kind):
   html = requests.get(url)
   soup = BeautifulSoup(html.text, 'html.parser')
   
   ###可能沒有分頁處理
   try:
     ###先找到class=cnt_page的div後再把div中的第一個span取出來(=頁數)
     pages = int(soup.select('.cnt_page span')[0].text)
     print("共有"+ str(pages) + '頁')
     for i in range(1, pages+1):
       ###組成各個分頁的網址
       url_page = url + '&page=' + str(i).strip()
       print('第' + str(i) + '頁:' + url_page)
       showpage(url_page, kind)
       if i == 2:
         break
   except:
     print('第一頁:' + url)
     showpage(url, kind)   

def digit(num):
  if num<10:
    numstr = '0' + str(num)
  else:
    numstr = str(num)
  return numstr

###找到該分頁中每本書的資訊並存入list中
###url:分頁的網址
def showpage(url ,kind):
   html = requests.get(url)
   soup = BeautifulSoup(html.text, 'html.parser')
   ###取得class=mod type02_m012 clearfix的第一個div區塊
   res = soup.find_all('div', {'class':'mod type02_m012 clearfix'})[0]
   ###取得所有class=item的div區塊
   items = res.select('.item')
   n=0
   for item in items:
     ###取得class=msg的第一個div區塊
     msg = item.select('.msg')[0]
     ###其中的3個'a'標籤
     title = msg.select('a')[0].text
     author = msg.select('a')[1].text
     publish = msg.select('a')[2].text
     ###取得span內容後用:把出版日期分開取出
     date = msg.find('span').text.split(":")[-1]
     
     ###取得'a'標籤中第一個'img'標籤裡的'src'屬性內容，此為圖片書本的資料
     src = item.select('a img')[0]["src"]
     ###取出圖片網址
     imgurl = src.split("?!=")[-1].split("&")[0]

     sale_price = item.select('.price .set2')[0].text
     ###將空白字元以空字串代替，同時濾除前後的字元
     content = item.select('.txt_cont')[0].text.replace(" ","").strip()
     
     listdata = [kind, title, imgurl, author, publish, date, sale_price, content]
     list1.append(listdata)

     n += 1
     print('查到的第' + str(n) + '本書')
     if n==3:
       break        

In [23]:
import requests
from bs4 import BeautifulSoup
import openpyxl

work = openpyxl.Workbook()
sheet = work.worksheets[0]
list1 = []


url = 'https://www.books.com.tw/web/books_nbtopm_01/?v=1&o=5'
urlhead = 'https://www.books.com.tw/web/books_nbtopm_'
###顯示模式為直式，排序依照暢銷度
urltail = '/?v=1&o=5'

###計算第幾個分類
num = 1

html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')
res = soup.find('div', {'class':'mod_b type02_l001-1 clearfix'})

###取得res中所有的'a'標籤
output = res.select("a")
#output[-1]  ###<a href="http://www.books.com.tw/web/books_nbtopm_20/?loc=P_0003_021">專業/教科書/政府出版品</a>

###只針對想要的類別去取數據(太多的話跑到一半會抓不到資料)
interested = ['輕小說', '文學小說', '商業理財', '旅遊', '電腦資訊']
for out in output:
   kind = out.text
   if kind in interested:
      ###因為分類的類別不是按照數字順序排列，因此直接從href中去取網址
      classify = out["href"]
      ###組成所有不同書類別的網址
      kindurl = urlhead + classify[41:43] + urltail
      print('第' + str(num) + '個分類:' + kind)
      showbook(kindurl,kind)
      num += 1
   else:
     continue   

listtitle = ['分類','書名','圖片網址','作者','出版社','出版日期','優惠價','內容']

sheet.append(listtitle)
for data in list1:
  sheet.append(data)

path = '/content/drive/MyDrive/Colab Notebooks/big data sample/book ranking/'
work.save(path + 'books_5.xlsx')

第1個分類:文學小說
共有6頁
第1頁:https://www.books.com.tw/web/books_nbtopm_01/?v=1&o=5&page=1
查到的第1本書
查到的第2本書
查到的第3本書
第2頁:https://www.books.com.tw/web/books_nbtopm_01/?v=1&o=5&page=2
查到的第1本書
查到的第2本書
查到的第3本書
第2個分類:商業理財
共有3頁
第1頁:https://www.books.com.tw/web/books_nbtopm_02/?v=1&o=5&page=1
查到的第1本書
查到的第2本書
查到的第3本書
第2頁:https://www.books.com.tw/web/books_nbtopm_02/?v=1&o=5&page=2
查到的第1本書
查到的第2本書
查到的第3本書
第3個分類:旅遊
第一頁:https://www.books.com.tw/web/books_nbtopm_11/?v=1&o=5
查到的第1本書
查到的第2本書
查到的第3本書
第4個分類:輕小說
共有2頁
第1頁:https://www.books.com.tw/web/books_nbtopm_15/?v=1&o=5&page=1
查到的第1本書
查到的第2本書
查到的第3本書
第2頁:https://www.books.com.tw/web/books_nbtopm_15/?v=1&o=5&page=2
查到的第1本書
查到的第2本書
查到的第3本書
第5個分類:電腦資訊
第一頁:https://www.books.com.tw/web/books_nbtopm_19/?v=1&o=5
查到的第1本書
查到的第2本書
查到的第3本書


In [24]:
output

[<a href="http://www.books.com.tw/web/books_nbtopm_01/?loc=P_0003_001">文學小說</a>,
 <a href="http://www.books.com.tw/web/books_nbtopm_02/?loc=P_0003_002">商業理財</a>,
 <a href="http://www.books.com.tw/web/books_nbtopm_03/?loc=P_0003_003">藝術設計</a>,
 <a href="http://www.books.com.tw/web/books_nbtopm_04/?loc=P_0003_004">人文社科</a>,
 <a href="http://www.books.com.tw/web/books_nbtopm_07/?loc=P_0003_005">心理勵志</a>,
 <a href="http://www.books.com.tw/web/books_nbtopm_12/?loc=P_0003_006">宗教命理</a>,
 <a href="http://www.books.com.tw/web/books_nbtopm_06/?loc=P_0003_007">自然科普</a>,
 <a href="http://www.books.com.tw/web/books_nbtopm_08/?loc=P_0003_008">醫療保健</a>,
 <a href="http://www.books.com.tw/web/books_nbtopm_09/?loc=P_0003_009">飲食</a>,
 <a href="http://www.books.com.tw/web/books_nbtopm_10/?loc=P_0003_010">生活風格</a>,
 <a href="http://www.books.com.tw/web/books_nbtopm_11/?loc=P_0003_011">旅遊</a>,
 <a href="http://www.books.com.tw/web/books_nbtopm_14/?loc=P_0003_012">童書/青少年文學</a>,
 <a href="http://www.books.c